<a href="https://colab.research.google.com/github/thimotyb/spark-notebooks/blob/main/mlflow_quickstart_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Compare runs, choose a model, and deploy it to a REST API
- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLflow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform


https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html



In [ ]:
!pip install tensorflow
!pip install hyperopt

In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip list

Package                            Version
---------------------------------- -----------
absl-py                            2.1.0
alembic                            1.13.3
asttokens                          2.4.1
astunparse                         1.6.3
blinker                            1.8.2
boltons                            23.0.0
brotlipy                           0.7.0
cachetools                         5.5.0
certifi                            2024.8.30
cffi                               1.15.1
charset-normalizer                 2.0.4
click                              8.1.7
cloudpickle                        3.1.0
comm                               0.2.2
conda                              23.9.0
conda-content-trust                0.2.0
conda-libmamba-solver              23.9.1
conda-package-handling             2.2.0
conda_package_streaming            0.9.0
contourpy                          1.3.0
cryptography                       41.0.3
cycler                             0.12

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


2024-10-29 14:55:03.270529: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 14:55:03.273282: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 14:55:03.280828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730213703.293983   96192 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730213703.299337   96192 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 14:55:03.313560: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [4]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [5]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result



In [6]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [8]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/10/29 15:36:25 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 14s 316ms/step - loss: 35.5569 - root_mean_squared_error: 5.9630
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2047 - root_mean_squared_error: 6.0168
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.3148 - root_mean_squared_error: 6.0260
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2769 - root_mean_squared_error: 6.0229
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2679 - root_mean_squared_error: 6.0222   
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.1233 - root_mean_squared_error: 6.0101   
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2905 - root_mean_squared_error: 6.0222   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.1402 - root_mean_squared_error: 5.0746 - val_loss: 4.9379 - val_root_mean_squared_error: 2.2221

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 134ms/step - loss: 5.0584 - root_mean_squared_error: 2.1914
 3/46 ━━━━━━━━━━━━

2024/10/29 15:36:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-ray-85 at: http://127.0.0.1:8080/#/experiments/145921017489783194/runs/2763fe5ace234f91ae1aae17f3d809e7.

2024/10/29 15:36:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/145921017489783194.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 14s 331ms/step - loss: 35.8275 - root_mean_squared_error: 5.9856
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.9401 - root_mean_squared_error: 5.6410
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.7302 - root_mean_squared_error: 4.7767   
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.5872 - root_mean_squared_error: 5.0256   
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.4437 - root_mean_squared_error: 4.7767   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.0273 - root_mean_squared_error: 2.4861 - val_loss: 0.8611 - val_root_mean_squared_error: 0.9279

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1224 - root_mean_squared_error: 1.0594
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0286 - root_mean_squared_error: 1.0131
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0066 - ro

2024/10/29 15:36:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run overjoyed-turtle-356 at: http://127.0.0.1:8080/#/experiments/145921017489783194/runs/e3910d471299404b9214edbfc83d4416.

2024/10/29 15:36:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/145921017489783194.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 14s 318ms/step - loss: 39.0739 - root_mean_squared_error: 6.2509
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39.6947 - root_mean_squared_error: 6.2979
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39.7179 - root_mean_squared_error: 6.2983
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39.6655 - root_mean_squared_error: 6.3021
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.6697 - root_mean_squared_error: 6.2983
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.0303 - root_mean_squared_error: 5.9139 - val_loss: 21.3704 - val_root_mean_squared_error: 4.6228

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 21.1518 - root_mean_squared_error: 4.6045
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.2536 - root_mean_squared_error: 4.6183
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21.2711 - root_m

2024/10/29 15:36:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run bedecked-moose-922 at: http://127.0.0.1:8080/#/experiments/145921017489783194/runs/2dde3200ac974705b94a6346e703be77.

2024/10/29 15:36:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/145921017489783194.



100%|██████████| 8/8 [00:34<00:00,  4.29s/trial, best loss: 0.7635937333106995]


2024/10/29 15:37:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run inquisitive-doe-654 at: http://127.0.0.1:8080/#/experiments/145921017489783194/runs/74f79ae263164e92a07a4b06fb16f04c.
2024/10/29 15:37:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/145921017489783194.


Best parameters: {'lr': np.float64(0.024572601179046992), 'momentum': np.float64(0.5928213475871887)}
Best eval rmse: 0.7635937333106995


- Open MLFlow UI and Inspect Rsults at http://localhost:8080
- Chart View
- Register the Model

- Serve the Model Locally: (in terminal)
curl https://pyenv.run | bash
pip install virtualenv
export MLFLOW_TRACKING_URI=http://localhost:8080
mlflow models serve -m "models:/wine-quality/1" --port 5002

- To test the model:
curl -d '{"dataframe_split": {
"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
"data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' \
-H 'Content-Type: application/json' -X POST localhost:5002/invocations



In [ ]:
- Build a model:
export MLFLOW_TRACKING_URI=http://localhost:8080
mlflow models build-docker --model-uri "models:/wine-quality/1" --name "qs_mlops"
docker run -p 5002:8080 qs_mlops
curl -d '{"dataframe_split": {"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' -H 'Content-Type: application/json' -X POST localhost:5002/invocations

